In [2]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

# Statistics of US Businesses  
Census Bureau Product, data download homepage is here: https://www.census.gov/programs-surveys/susb/data/tables.html  
Currently using county by state 2020: https://www.census.gov/data/tables/2020/econ/susb/2020-susb-annual.html  
This is for 2020 released in 2023.  
When downloaded, open the Excel workbook and delete the top two rows (the title and the notes) so that we are left with the row of column headers at the top. Copy-paste the content into an unformatted csv, and save in Data Downloads as "StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2020.csv", etc. for different years. When you go back to 2015 there is both establishment and enterprise industry, select establishment industry. For US, "US & States, NAICS, detailed employment size".

In [121]:
twentytwenty = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2020.csv', dtype = 'str')
twentytwenty = twentytwenty.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag'])
twentytwenty = twentytwenty.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentytwenty['Year'] = '2020'
twentytwenty['State Name'] = twentytwenty['State Name'].str.capitalize()
twentytwenty['County Name'] = twentytwenty['County Name'].str.capitalize()
twentytwenty = twentytwenty.loc[twentytwenty['State'] == '47']
twentytwenty.head(2)

,State,State Name,County,County Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year
390347,47,Tennessee,001,Anderson,--,Total,1: Total,1305,1532,42762,2831834,2020
390348,47,Tennessee,001,Anderson,--,Total,2: <20 employees,861,870,3797,146123,2020


In [122]:
twentytwenty['County Name'].unique()

array(['Anderson', 'Bedford', 'Benton', 'Bledsoe', 'Blount', 'Bradley',
       'Campbell', 'Cannon', 'Carroll', 'Carter', 'Cheatham', 'Chester',
       'Claiborne', 'Clay', 'Cocke', 'Coffee', 'Crockett', 'Cumberland',
       'Davidson', 'Decatur', 'Dekalb', 'Dickson', 'Dyer', 'Fayette',
       'Fentress', 'Franklin', 'Gibson', 'Giles', 'Grainger', 'Greene',
       'Grundy', 'Hamblen', 'Hamilton', 'Hancock', 'Hardeman', 'Hardin',
       'Hawkins', 'Haywood', 'Henderson', 'Henry', 'Hickman', 'Houston',
       'Humphreys', 'Jackson', 'Jefferson', 'Johnson', 'Knox', 'Lake',
       'Lauderdale', 'Lawrence', 'Lewis', 'Lincoln', 'Loudon', 'Mcminn',
       'Mcnairy', 'Macon', 'Madison', 'Marion', 'Marshall', 'Maury',
       'Meigs', 'Monroe', 'Montgomery', 'Moore', 'Morgan', 'Obion',
       'Overton', 'Perry', 'Pickett', 'Polk', 'Putnam', 'Rhea', 'Roane',
       'Robertson', 'Rutherford', 'Scott', 'Sequatchie', 'Sevier',
       'Shelby', 'Smith', 'Stewart', 'Sullivan', 'Sumner', 'Tipton',
    

In [88]:
twentytwentyb = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_USStates_2020.csv', dtype = 'str')
twentytwentyb = twentytwentyb.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag'])
twentytwentyb = twentytwentyb.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentytwentyb['Year'] = '2020'
twentytwentyb['County'] = 'All'
twentytwentyb['County Name'] = 'All'
twentytwentyb = twentytwentyb.loc[(twentytwentyb['State Name'] == 'United States')|(twentytwentyb['State Name'] == 'Tennessee')].reset_index(drop = True)
twentytwentyb['State Name'] = twentytwentyb['State Name'].str.capitalize()
twentytwentyb['County Name'] = twentytwentyb['County Name'].str.capitalize()
twentytwentyb.tail(2)

,State,State Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,County,County Name
44381,47,Tennessee,99,Industries not classified,06: <20 employees,235,235,395,10683,2020,All,All
44382,47,Tennessee,99,Industries not classified,19: <500 employees,236,236,425,12604,2020,All,All


In [149]:
thelist = ['01: Total', '02: <5 employees', '03: 5-9 employees', '04:10-14 employees', '05: 15-19 employees']
test = twentytwentyb.loc[(twentytwentyb['Enterprise Size'].isin(thelist))&(twentytwentyb['NAICS Description'] == 'Total')]
test

,State,State Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,County,County Name
0,00,United states,--,Total,01: Total,6140612,8000178,134163349,7564809878,2020,All,All
1,00,United states,--,Total,02: <5 employees,3828482,3836380,6018364,295178934,2020,All,All
2,00,United states,--,Total,03: 5-9 employees,1004591,1017938,6619736,264502785,2020,All,All
3,00,United states,--,Total,04:10-14 employees,416089,432961,4871463,199198898,2020,All,All
4,00,United states,--,Total,05: 15-19 employees,222574,238558,3732378,154030669,2020,All,All
43906,47,Tennessee,--,Total,01: Total,101824,140905,2760605,133051786,2020,All,All
43907,47,Tennessee,--,Total,02: <5 employees,54072,54184,93437,4116099,2020,All,All
43908,47,Tennessee,--,Total,03: 5-9 employees,18117,18304,119070,4470941,2020,All,All
43909,47,Tennessee,--,Total,04:10-14 employees,7529,7839,87726,3409600,2020,All,All
43910,47,Tennessee,--,Total,05: 15-19 employees,4166,4441,69125,2803694,2020,All,All


In [119]:
twentynineteen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2019.csv', dtype = 'str')
twentynineteen = twentynineteen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag'])
twentynineteen = twentynineteen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentynineteen['Year'] = '2019'
twentynineteen['State Name'] = twentynineteen['State Name'].str.capitalize()
twentynineteen['County Name'] = twentynineteen['County Name'].str.capitalize()
twentynineteen = twentynineteen.loc[twentynineteen['State'] == '47']
twentynineteen.head(2)

,State,State Name,County,County Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year
391343,47,Tennessee,001,Anderson,--,Total,1: Total,1296,1535,41120,2576321,2019
391344,47,Tennessee,001,Anderson,--,Total,2: <20 employees,856,865,3810,143502,2019


In [120]:
twentynineteen['County Name'].unique()

array(['Anderson', 'Bedford', 'Benton', 'Bledsoe', 'Blount', 'Bradley',
       'Campbell', 'Cannon', 'Carroll', 'Carter', 'Cheatham', 'Chester',
       'Claiborne', 'Clay', 'Cocke', 'Coffee', 'Crockett', 'Cumberland',
       'Davidson', 'Decatur', 'Dekalb', 'Dickson', 'Dyer', 'Fayette',
       'Fentress', 'Franklin', 'Gibson', 'Giles', 'Grainger', 'Greene',
       'Grundy', 'Hamblen', 'Hamilton', 'Hancock', 'Hardeman', 'Hardin',
       'Hawkins', 'Haywood', 'Henderson', 'Henry', 'Hickman', 'Houston',
       'Humphreys', 'Jackson', 'Jefferson', 'Johnson', 'Knox', 'Lake',
       'Lauderdale', 'Lawrence', 'Lewis', 'Lincoln', 'Loudon', 'Mcminn',
       'Mcnairy', 'Macon', 'Madison', 'Marion', 'Marshall', 'Maury',
       'Meigs', 'Monroe', 'Montgomery', 'Moore', 'Morgan', 'Obion',
       'Overton', 'Perry', 'Pickett', 'Polk', 'Putnam', 'Rhea', 'Roane',
       'Robertson', 'Rutherford', 'Scott', 'Sequatchie', 'Sevier',
       'Shelby', 'Smith', 'Stewart', 'Sullivan', 'Sumner', 'Tipton',
    

In [146]:
twentynineteenb = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_USStates_2019.csv', dtype = 'str')
twentynineteenb = twentynineteenb.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag'])
twentynineteenb = twentynineteenb.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentynineteenb['Year'] = '2019'
twentynineteenb['County'] = 'All'
twentynineteenb['County Name'] = 'All'
twentynineteenb = twentynineteenb.loc[(twentynineteenb['State Name'] == 'United States')|(twentynineteenb['State Name'] == 'Tennessee')].reset_index(drop = True)
twentynineteenb['State Name'] = twentynineteenb['State Name'].str.capitalize()
twentynineteenb['County Name'] = twentynineteenb['County Name'].str.capitalize()
twentynineteenb.head(5)

,State,State Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,County,County Name
0,00,United states,--,Total,01: Total,6102412,7959103,132989428,7428553593,2019,All,All
1,00,United states,--,Total,02: <5 employees,3777085,3783930,6003770,294917415,2019,All,All
2,00,United states,--,Total,03: 5-9 employees,1013629,1026898,6681959,267500052,2019,All,All
3,00,United states,--,Total,04:10-14 employees,417365,434240,4884899,202137177,2019,All,All
4,00,United states,--,Total,05: 15-19 employees,223462,239895,3747797,156834000,2019,All,All


In [148]:
thelist = ['01: Total', '02: <5 employees', '03: 5-9 employees', '04:10-14 employees', '05: 15-19 employees']
test = twentynineteenb.loc[(twentynineteenb['Enterprise Size'].isin(thelist))&(twentynineteenb['NAICS Description'] == 'Total')]
test

,State,State Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,County,County Name
0,00,United states,--,Total,01: Total,6102412,7959103,132989428,7428553593,2019,All,All
1,00,United states,--,Total,02: <5 employees,3777085,3783930,6003770,294917415,2019,All,All
2,00,United states,--,Total,03: 5-9 employees,1013629,1026898,6681959,267500052,2019,All,All
3,00,United states,--,Total,04:10-14 employees,417365,434240,4884899,202137177,2019,All,All
4,00,United states,--,Total,05: 15-19 employees,223462,239895,3747797,156834000,2019,All,All
43980,47,Tennessee,--,Total,01: Total,100734,139760,2724545,129928960,2019,All,All
43981,47,Tennessee,--,Total,02: <5 employees,53034,53132,93146,4020689,2019,All,All
43982,47,Tennessee,--,Total,03: 5-9 employees,18101,18308,119322,4390330,2019,All,All
43983,47,Tennessee,--,Total,04:10-14 employees,7593,7893,88196,3358602,2019,All,All
43984,47,Tennessee,--,Total,05: 15-19 employees,4130,4378,68189,2739672,2019,All,All


In [114]:
twentyeighteen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2018.csv', dtype = 'str')
twentyeighteen = twentyeighteen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag'])
twentyeighteen = twentyeighteen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentyeighteen['Year'] = '2018'
twentyeighteen['State Name'] = twentyeighteen['State Name'].str.capitalize()
twentyeighteen['County Name'] = twentyeighteen['County Name'].str.capitalize()
twentyeighteen = twentyeighteen.loc[twentyeighteen['State'] == '47']
twentyeighteen.head(2)

,State,State Name,County,County Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year
391789,47,Tennessee,001,Anderson,--,Total,1: Total,1315,1545,40846,2492519,2018
391790,47,Tennessee,001,Anderson,--,Total,2: <20 employees,882,891,4117,150623,2018


In [115]:
twentyeighteen['County Name'].unique()

array(['Anderson', 'Bedford', 'Benton', 'Bledsoe', 'Blount', 'Bradley',
       'Campbell', 'Cannon', 'Carroll', 'Carter', 'Cheatham', 'Chester',
       'Claiborne', 'Clay', 'Cocke', 'Coffee', 'Crockett', 'Cumberland',
       'Davidson', 'Decatur', 'Dekalb', 'Dickson', 'Dyer', 'Fayette',
       'Fentress', 'Franklin', 'Gibson', 'Giles', 'Grainger', 'Greene',
       'Grundy', 'Hamblen', 'Hamilton', 'Hancock', 'Hardeman', 'Hardin',
       'Hawkins', 'Haywood', 'Henderson', 'Henry', 'Hickman', 'Houston',
       'Humphreys', 'Jackson', 'Jefferson', 'Johnson', 'Knox', 'Lake',
       'Lauderdale', 'Lawrence', 'Lewis', 'Lincoln', 'Loudon', 'Mcminn',
       'Mcnairy', 'Macon', 'Madison', 'Marion', 'Marshall', 'Maury',
       'Meigs', 'Monroe', 'Montgomery', 'Moore', 'Morgan', 'Obion',
       'Overton', 'Perry', 'Pickett', 'Polk', 'Putnam', 'Rhea', 'Roane',
       'Robertson', 'Rutherford', 'Scott', 'Sequatchie', 'Sevier',
       'Shelby', 'Smith', 'Stewart', 'Sullivan', 'Sumner', 'Tipton',
    

In [92]:
twentyeighteenb = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_USStates_2018.csv', dtype = 'str')
twentyeighteenb = twentyeighteenb.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag'])
twentyeighteenb = twentyeighteenb.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentyeighteenb['Year'] = '2018'
twentyeighteenb['County'] = 'All'
twentyeighteenb['County Name'] = 'All'
twentyeighteenb = twentyeighteenb.loc[(twentyeighteenb['State Name'] == 'United States')|(twentyeighteenb['State Name'] == 'Tennessee')].reset_index(drop = True)
twentyeighteenb['State Name'] = twentyeighteenb['State Name'].str.capitalize()
twentyeighteenb['County Name'] = twentyeighteenb['County Name'].str.capitalize()
twentyeighteenb.tail(2)

,State,State Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,County,County Name
44356,47,Tennessee,99,Industries not classified,06: <20 employees,224,224,286,7682,2018,All,All
44357,47,Tennessee,99,Industries not classified,19: <500 employees,225,225,304,7907,2018,All,All


In [117]:
#this year they started including "Employment Range Flag" so drop that column
twentyseventeen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2017.csv', dtype = 'str')
twentyseventeen = twentyseventeen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag', 
                                                  'Receipts\n($1,000)', 'Receipts\nNoise Flag'])
twentyseventeen = twentyseventeen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentyseventeen['Year'] = '2017'
twentyseventeen['State Name'] = twentyseventeen['State Name'].str.capitalize()
twentyseventeen['County Name'] = twentyseventeen['County Name'].str.capitalize()
twentyseventeen = twentyseventeen.loc[twentyseventeen['State'] == '47']
twentyseventeen.head(2)

,State,State Name,County,County Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year
392733,47,Tennessee,001,Anderson,--,Total,1: Total,1333,1575,40706,2457011,2017
392734,47,Tennessee,001,Anderson,--,Total,2: <20 employees,900,907,4001,153308,2017


In [118]:
twentyseventeen['County Name'].unique()

array(['Anderson', 'Bedford', 'Benton', 'Bledsoe', 'Blount', 'Bradley',
       'Campbell', 'Cannon', 'Carroll', 'Carter', 'Cheatham', 'Chester',
       'Claiborne', 'Clay', 'Cocke', 'Coffee', 'Crockett', 'Cumberland',
       'Davidson', 'Decatur', 'Dekalb', 'Dickson', 'Dyer', 'Fayette',
       'Fentress', 'Franklin', 'Gibson', 'Giles', 'Grainger', 'Greene',
       'Grundy', 'Hamblen', 'Hamilton', 'Hancock', 'Hardeman', 'Hardin',
       'Hawkins', 'Haywood', 'Henderson', 'Henry', 'Hickman', 'Houston',
       'Humphreys', 'Jackson', 'Jefferson', 'Johnson', 'Knox', 'Lake',
       'Lauderdale', 'Lawrence', 'Lewis', 'Lincoln', 'Loudon', 'Mcminn',
       'Mcnairy', 'Macon', 'Madison', 'Marion', 'Marshall', 'Maury',
       'Meigs', 'Monroe', 'Montgomery', 'Moore', 'Morgan', 'Obion',
       'Overton', 'Perry', 'Pickett', 'Polk', 'Putnam', 'Rhea', 'Roane',
       'Robertson', 'Rutherford', 'Scott', 'Sequatchie', 'Sevier',
       'Shelby', 'Smith', 'Stewart', 'Sullivan', 'Sumner', 'Tipton',
    

In [94]:
#this year they started including "Employment Range Flag" so drop that column
twentyseventeenb = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_USStates_2017.csv', dtype = 'str')
twentyseventeenb = twentyseventeenb.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment\nRange Flag', 
                                                  'Receipts\n($1,000)', 'Receipts\nNoise Flag'])
twentyseventeenb = twentyseventeenb.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentyseventeenb['Year'] = '2017'
twentyseventeenb['County'] = 'All'
twentyseventeenb['County Name'] = 'All'
twentyseventeenb = twentyseventeenb.loc[(twentyseventeenb['State Name'] == 'United States')|(twentyseventeenb['State Name'] == 'Tennessee')].reset_index(drop = True)
twentyseventeenb['State Name'] = twentyseventeenb['State Name'].str.capitalize()
twentyseventeenb['County Name'] = twentyseventeenb['County Name'].str.capitalize()
twentyseventeenb.tail(2)

,State,State Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,County,County Name
44308,47,Tennessee,99,Industries not classified,06: <20 employees,140,140,186,4223,2017,All,All
44309,47,Tennessee,99,Industries not classified,19: <500 employees,142,142,241,4733,2017,All,All


In [123]:
# #holding on downloading past 2017 for us because the file format is broken up with large and small businesses.. waiting till relevant request
# twentysixteen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2016.csv', dtype = 'str')
# twentysixteen = twentysixteen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twentysixteen = twentysixteen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twentysixteen['Year'] = '2016'
# twentysixteen.head(2)

In [124]:
# twentyfifteen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2015.csv', dtype = 'str')
# twentyfifteen = twentyfifteen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employmment Range Flag'])
# twentyfifteen = twentyfifteen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twentyfifteen['Year'] = '2015'
# twentyfifteen.head(2)

## They started doing NAICS breakouts at the county level in 2015, so from here back it's only total

In [97]:
# twentyfourteen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2014.csv', dtype = 'str')
# twentyfourteen = twentyfourteen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twentyfourteen = twentyfourteen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twentyfourteen['Year'] = '2014'
# twentyfourteen.head(2)

In [98]:
# twentythirteen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2013.csv', dtype = 'str')
# twentythirteen = twentythirteen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twentythirteen = twentythirteen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twentythirteen['Year'] = '2013'
# twentythirteen.head(2)

In [99]:
# twentytwelve = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2012.csv', dtype = 'str')
# twentytwelve = twentytwelve.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twentytwelve = twentytwelve.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twentytwelve['Year'] = '2012'
# twentytwelve.head(2)

In [100]:
# twentyeleven = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2011.csv', dtype = 'str')
# twentyeleven = twentyeleven.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twentyeleven = twentyeleven.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twentyeleven['Year'] = '2011'
# twentyeleven.head(2)

In [101]:
# twentyten = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2010.csv', dtype = 'str')
# twentyten = twentyten.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twentyten = twentyten.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twentyten['Year'] = '2010'
# twentyten.head(2)

In [102]:
# twothounine = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2009.csv', dtype = 'str')
# twothounine = twothounine.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twothounine = twothounine.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twothounine['Year'] = '2009'
# twothounine.head(2)

In [103]:
# twothoueight = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2008.csv', dtype = 'str')
# twothoueight = twothoueight.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twothoueight = twothoueight.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twothoueight['Year'] = '2008'
# twothoueight.head(2)

In [104]:
# twothouseven = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2007.csv', dtype = 'str')
# twothouseven = twothouseven.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag', 
#                                             'Receipts', 'Receipts Noise Flag'])
# twothouseven = twothouseven.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twothouseven['Year'] = '2007'
# twothouseven.head(2)

In [105]:
# twothousix = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2006.csv', dtype = 'str')
# #twothousix = twothousix.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# #twothousix = twothousix.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twothousix['Year'] = '2006'
# twothousix.head(2)

## Merging all dataframes with the NAICS breakout, read in whatever you need later.

In [125]:
dfs = [twentytwenty, twentynineteen, twentyeighteen, twentyseventeen]

In [126]:
data = pd.concat(dfs).reset_index(drop = True)

In [127]:
data.head()

,State,State Name,County,County Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year
0,47,Tennessee,001,Anderson,--,Total,1: Total,1305,1532,42762,2831834,2020
1,47,Tennessee,001,Anderson,--,Total,2: <20 employees,861,870,3797,146123,2020
2,47,Tennessee,001,Anderson,--,Total,3: 20-99 employees,145,165,5113,250992,2020
3,47,Tennessee,001,Anderson,--,Total,4: 100-499 employees,70,112,3613,202320,2020
4,47,Tennessee,001,Anderson,--,Total,5: 500+ employees,229,385,30239,2232399,2020


In [128]:
data = data.loc[data['State'] == '47'].reset_index(drop = True)

In [129]:
data['County Name'].unique()

array(['Anderson', 'Bedford', 'Benton', 'Bledsoe', 'Blount', 'Bradley',
       'Campbell', 'Cannon', 'Carroll', 'Carter', 'Cheatham', 'Chester',
       'Claiborne', 'Clay', 'Cocke', 'Coffee', 'Crockett', 'Cumberland',
       'Davidson', 'Decatur', 'Dekalb', 'Dickson', 'Dyer', 'Fayette',
       'Fentress', 'Franklin', 'Gibson', 'Giles', 'Grainger', 'Greene',
       'Grundy', 'Hamblen', 'Hamilton', 'Hancock', 'Hardeman', 'Hardin',
       'Hawkins', 'Haywood', 'Henderson', 'Henry', 'Hickman', 'Houston',
       'Humphreys', 'Jackson', 'Jefferson', 'Johnson', 'Knox', 'Lake',
       'Lauderdale', 'Lawrence', 'Lewis', 'Lincoln', 'Loudon', 'Mcminn',
       'Mcnairy', 'Macon', 'Madison', 'Marion', 'Marshall', 'Maury',
       'Meigs', 'Monroe', 'Montgomery', 'Moore', 'Morgan', 'Obion',
       'Overton', 'Perry', 'Pickett', 'Polk', 'Putnam', 'Rhea', 'Roane',
       'Robertson', 'Rutherford', 'Scott', 'Sequatchie', 'Sevier',
       'Shelby', 'Smith', 'Stewart', 'Sullivan', 'Sumner', 'Tipton',
    

In [130]:
dfs = [data, twentytwentyb, twentynineteenb, twentyeighteenb, twentyseventeenb]
data = pd.concat(dfs)

In [133]:
data['State Name'].unique()

array(['Tennessee', 'United states'], dtype=object)

In [134]:
data = data.rename(columns = {'State': 'StateFIPS', 'County': 'CountyFIPS', 'County Name': 'County'})

In [135]:
data['GEO_ID'] = '0500000US' + data['StateFIPS'] + data['CountyFIPS']

In [136]:
# data = data.drop(columns = ['StateFIPS', 'State Name', 'CountyFIPS'])

In [137]:
naicsdict = dict(zip(data['NAICS'], data['NAICS Description']))
countydict = dict(zip(data['County'], data['GEO_ID']))

In [138]:
data.head()

,StateFIPS,State Name,CountyFIPS,County,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,GEO_ID
0,47,Tennessee,001,Anderson,--,Total,1: Total,1305,1532,42762,2831834,2020,0500000US47001
1,47,Tennessee,001,Anderson,--,Total,2: <20 employees,861,870,3797,146123,2020,0500000US47001
2,47,Tennessee,001,Anderson,--,Total,3: 20-99 employees,145,165,5113,250992,2020,0500000US47001
3,47,Tennessee,001,Anderson,--,Total,4: 100-499 employees,70,112,3613,202320,2020,0500000US47001
4,47,Tennessee,001,Anderson,--,Total,5: 500+ employees,229,385,30239,2232399,2020,0500000US47001


In [139]:
# data = data.set_index(['County Name', ''])

In [142]:
data['County'].unique()

array(['Anderson', 'Bedford', 'Benton', 'Bledsoe', 'Blount', 'Bradley',
       'Campbell', 'Cannon', 'Carroll', 'Carter', 'Cheatham', 'Chester',
       'Claiborne', 'Clay', 'Cocke', 'Coffee', 'Crockett', 'Cumberland',
       'Davidson', 'Decatur', 'Dekalb', 'Dickson', 'Dyer', 'Fayette',
       'Fentress', 'Franklin', 'Gibson', 'Giles', 'Grainger', 'Greene',
       'Grundy', 'Hamblen', 'Hamilton', 'Hancock', 'Hardeman', 'Hardin',
       'Hawkins', 'Haywood', 'Henderson', 'Henry', 'Hickman', 'Houston',
       'Humphreys', 'Jackson', 'Jefferson', 'Johnson', 'Knox', 'Lake',
       'Lauderdale', 'Lawrence', 'Lewis', 'Lincoln', 'Loudon', 'Mcminn',
       'Mcnairy', 'Macon', 'Madison', 'Marion', 'Marshall', 'Maury',
       'Meigs', 'Monroe', 'Montgomery', 'Moore', 'Morgan', 'Obion',
       'Overton', 'Perry', 'Pickett', 'Polk', 'Putnam', 'Rhea', 'Roane',
       'Robertson', 'Rutherford', 'Scott', 'Sequatchie', 'Sevier',
       'Shelby', 'Smith', 'Stewart', 'Sullivan', 'Sumner', 'Tipton',
    

In [144]:
test = data.loc[(data['State Name'] == 'Tennessee')]
test.head()

,StateFIPS,State Name,CountyFIPS,County,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,GEO_ID
0,00,United states,All,All,--,Total,01: Total,6140612,8000178,134163349,7564809878,2020,0500000US00All
1,00,United states,All,All,--,Total,02: <5 employees,3828482,3836380,6018364,295178934,2020,0500000US00All
2,00,United states,All,All,--,Total,03: 5-9 employees,1004591,1017938,6619736,264502785,2020,0500000US00All
3,00,United states,All,All,--,Total,04:10-14 employees,416089,432961,4871463,199198898,2020,0500000US00All
4,00,United states,All,All,--,Total,05: 15-19 employees,222574,238558,3732378,154030669,2020,0500000US00All


In [140]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('SUSB_Annual_2015to2020', conn, if_exists = 'replace', index = False)

234011